<a href="https://colab.research.google.com/github/sssteeefaaan/VIII-Semestar/blob/main/Paralelni%20Sistemi/Laboratorijske%20Ve%C5%BEbe/Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalacija

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

!git clone https://github.com/andreinechaev/nvcc4jupyter.git

!pip install ./nvcc4jupyter/

%load_ext nvcc_plugin

# Zadatak 1

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji računa sledeći izraz: A * x + B, gde su A i B vektori, a x skalar. Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja programa u zavisnosti od broja blokova i broja niti u okviru jednog bloka.
%%writefile zadatak1.cu

#include <stdio.h>
#define N 5120
#define BLOCK_SIZE 32

__const__ int x = 5;

__host__ void operate_on_GPU(int *A, int *B, int *C);
__global__ void operate(int *A, int *B, int *C);
__host__ bool check_result(int *A, int *B, int *GPU_C);
__device__ __host__ int operation(int a, int b) { return a * x + b; }

int main(int argc, char** argv)
{
    int *A, *B, *GPU_C;
    A = (int*) malloc(sizeof(int) * N);
    B = (int*) malloc(sizeof(int) * N);
    GPU_C = (int*) malloc(sizeof(int) * N);
 
    for(int i = 0; i < N; i++) {
        A[i] = i + 1;
        B[i] = N - (i + 1);
    }
 
    operate_on_GPU(A, B, GPU_C);
 
    printf("c = |\t");
    for(int i = 0; i < N; i++)
      printf("%d\t", GPU_C[i]);
    printf("|\n");
 
    if(check_result(A, B, GPU_C))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat je netacan!\n");
 
    free(GPU_C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *GPU_C)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
        rez &= (GPU_C[i] == operation(A[i], B[i]));
    return rez;
}

__host__ void operate_on_GPU(int *A, int *B, int *C)
{
    int *dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1);
 
    operate<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void operate(int *A, int* B, int *C)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x;

    for(int i = tid_x; i < N; i+= gridDim.x * blockDim.x)
        C[i] = operation(A[i], B[i]);
}

In [ ]:
#@title Compile and run
filepath = "zadatak1.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 2

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji računa sledeći izraz: A * x - B, gde su A i B vektori, a x skalar. Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja programa u zavisnosti od broja blokova i broja niti u okviru jednog bloka.
%%writefile zadatak2.cu

#include <stdio.h>
#define N 5120
#define BLOCK_SIZE 32

__const__ int x = 5;

__host__ void operate_on_GPU(int *A, int *B, int *C);
__global__ void operate(int *A, int *B, int *C);
__host__ bool check_result(int *A, int *B, int *GPU_C);
__device__ __host__ int operation(int a, int b) { return a * x - b; }

int main(int argc, char** argv)
{
    int *A, *B, *GPU_C;
    A = (int*) malloc(sizeof(int) * N);
    B = (int*) malloc(sizeof(int) * N);
    GPU_C = (int*) malloc(sizeof(int) * N);
 
    for(int i = 0; i < N; i++) {
        A[i] = i + 1;
        B[i] = N - (i + 1);
    }
 
    operate_on_GPU(A, B, GPU_C);
 
    printf("c = |\t");
    for(int i = 0; i < N; i++)
      printf("%d\t", GPU_C[i]);
    printf("|\n");
 
    if(check_result(A, B, GPU_C))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat je netacan!\n");
 
    free(GPU_C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *GPU_C)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
        rez &= (GPU_C[i] == operation(A[i], B[i]));
    return rez;
}

__host__ void operate_on_GPU(int *A, int *B, int *C)
{
    int *dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1);
 
    operate<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void operate(int *A, int* B, int *C)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x;

    for(int i = tid_x; i < N; i+= gridDim.x * blockDim.x)
        C[i] = operation(A[i], B[i]);
}

In [ ]:
#@title Compile and run
filepath = "zadatak2.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 3

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji računa sledeći izraz: A + B, gde su A i B kvadratne matrice Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja programa u zavisnosti od broja blokova i broja niti u okviru jednog bloka
%%writefile zadatak3.cu

#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32

__host__ void operate_on_GPU(int *A, int *B, int *C);
__global__ void operate(int *A, int *B, int *C);
__host__ bool check_result(int *A, int *B, int *GPU_C);
__device__ __host__ int operation(int a, int b) { return a + b; }

int main(int argc, char** argv)
{
    int *A, *B, *GPU_C;
    A = (int*) malloc(sizeof(int) * N * N);
    B = (int*) malloc(sizeof(int) * N * N);
    GPU_C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++) {
        for(int j = 0; j < N; j++) {
          A[i * N + j] = i * N + j + 1;
          B[i * N + j] = j * N + i + 1;
        }
    }
 
    operate_on_GPU(A, B, GPU_C);
 
    for(int i = 0; i < N; i++) {
      printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", GPU_C[i * N + j]);
      printf("|\n");
    }
 
    if(check_result(A, B, GPU_C))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat je netacan!\n");
 
    free(GPU_C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *GPU_C)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (GPU_C[i * N + j] == operation(A[i * N + j], B[i * N + j]));
    return rez;
}

__host__ void operate_on_GPU(int *A, int *B, int *C)
{
    int *dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.x + 1);
 
    operate<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void operate(int *A, int* B, int *C)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x,
        tid_y = blockIdx.y * blockDim.y + threadIdx.y;

    for(int i = tid_x; i < N; i+= gridDim.x * blockDim.x)
      for(int j = tid_y; j < N; j += gridDim.y * blockDim.y)
          C[i * N + j] = operation(A[i * N + j], B[i * N + j]);
}

In [ ]:
#@title Compile and run
filepath = "zadatak3.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 4

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji računa sledeći izraz: A - B, gde su A i B kvadratne matrice Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja programa u zavisnosti od broja blokova i broja niti u okviru jednog bloka
%%writefile zadatak4.cu

#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32

__host__ void operate_on_GPU(int *A, int *B, int *C);
__global__ void operate(int *A, int *B, int *C);
__host__ bool check_result(int *A, int *B, int *GPU_C);
__device__ __host__ int operation(int a, int b) { return a - b; }

int main(int argc, char** argv)
{
    int *A, *B, *GPU_C;
    A = (int*) malloc(sizeof(int) * N * N);
    B = (int*) malloc(sizeof(int) * N * N);
    GPU_C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++) {
        for(int j = 0; j < N; j++) {
          A[i * N + j] = i * N + j + 1;
          B[i * N + j] = j * N + i + 1;
        }
    }
 
    operate_on_GPU(A, B, GPU_C);
 
    for(int i = 0; i < N; i++) {
      printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", GPU_C[i * N + j]);
      printf("|\n");
    }
 
    if(check_result(A, B, GPU_C))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat je netacan!\n");
 
    free(GPU_C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *GPU_C)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (GPU_C[i * N + j] == operation(A[i * N + j], B[i * N + j]));
    return rez;
}

__host__ void operate_on_GPU(int *A, int *B, int *C)
{
    int *dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.x + 1);
 
    operate<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void operate(int *A, int* B, int *C)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x,
        tid_y = blockIdx.y * blockDim.y + threadIdx.y;

    for(int i = tid_x; i < N; i+= gridDim.x * blockDim.x)
      for(int j = tid_y; j < N; j += gridDim.y * blockDim.y)
          C[i * N + j] = operation(A[i * N + j], B[i * N + j]);
}

In [ ]:
#@title Compile and run
filepath = "zadatak4.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 5

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji računa sledeći izraz: A + B, gde su A i B vektori, i u novodobijenom vektoru sve vrednosti koje su veće od nekog k zameniti brojem p. Brojeve k i p unosi korisnik. Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju pnašanja  programa u zavisnosti od broja blokova i broja niti u okviru jednog bloka.
%%writefile zadatak5.cu

#include <stdio.h>
#define N 5120
#define BLOCK_SIZE 32

__host__ void operate_on_GPU(int *A, int *B, int *C, int k, int p);
__global__ void operate(int *A, int *B, int *C, int k, int p);
__host__ bool check_result(int *A, int *B, int *GPU_C, int k, int p);
__device__ __host__ int operation(int a, int b, int k, int p) { return a + b > k ? p : a + b; }

int main(int argc, char** argv)
{
    int *A, *B, *GPU_C, k, p;
    A = (int*) malloc(sizeof(int) * N);
    B = (int*) malloc(sizeof(int) * N);
    GPU_C = (int*) malloc(sizeof(int) * N);
 
    for(int i = 0; i < N; i++) {
        A[i] = i * i;
        B[i] = N - (i + 1);
    }
 
    if(argc == 3) {
        k = atoi(argv[1]);
        p = atoi(argv[2]);
    }
    else {
        k = 17090;
        p = 2904;
    }
 
    operate_on_GPU(A, B, GPU_C, k, p);
 
    printf("c = |\t");
    for(int i = 0; i < N; i++)
      printf("%d\t", GPU_C[i]);
    printf("|\n");
 
    if(check_result(A, B, GPU_C, k, p))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat je netacan!\n");
 
    free(GPU_C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *GPU_C, int k, int p)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
        rez &= (GPU_C[i] == operation(A[i], B[i], k, p));
    return rez;
}

__host__ void operate_on_GPU(int *A, int *B, int *C, int k, int p)
{
    int *dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1);
 
    operate<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C, k, p);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void operate(int *A, int* B, int *C, int k, int p)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x;

    for(int i = tid_x; i < N; i+= gridDim.x * blockDim.x)
        C[i] = operation(A[i], B[i], k, p);
}

In [ ]:
#@title Compile and run
filepath = "zadatak5.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "5674 8" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 6

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji računa sledeći izraz: A - B, gde su A i B vektori, i u novodobijenom vektoru sve vrednosti koje su manje od nekog k zameniti brojem p. Brojeve k i p unosi korisnik. Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju pnašanja  programa u zavisnosti od broja blokova i broja niti u okviru jednog bloka.
%%writefile zadatak6.cu

#include <stdio.h>
#define N 5120
#define BLOCK_SIZE 32

__host__ void operate_on_GPU(int *A, int *B, int *C, int k, int p);
__global__ void operate(int *A, int *B, int *C, int k, int p);
__host__ bool check_result(int *A, int *B, int *GPU_C, int k, int p);
__device__ __host__ int operation(int a, int b, int k, int p) { return a - b < k ? p : a - b; }

int main(int argc, char** argv)
{
    int *A, *B, *GPU_C, k, p;
    A = (int*) malloc(sizeof(int) * N);
    B = (int*) malloc(sizeof(int) * N);
    GPU_C = (int*) malloc(sizeof(int) * N);
 
    for(int i = 0; i < N; i++) {
        A[i] = i * i;
        B[i] = N - (i + 1);
    }
 
    if(argc == 3) {
        k = atoi(argv[1]);
        p = atoi(argv[2]);
    }
    else {
        k = 17090;
        p = 2904;
    }
 
    operate_on_GPU(A, B, GPU_C, k, p);
 
    printf("c = |\t");
    for(int i = 0; i < N; i++)
      printf("%d\t", GPU_C[i]);
    printf("|\n");
 
    if(check_result(A, B, GPU_C, k, p))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat je netacan!\n");
 
    free(GPU_C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *GPU_C, int k, int p)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
        rez &= (GPU_C[i] == operation(A[i], B[i], k, p));
    return rez;
}

__host__ void operate_on_GPU(int *A, int *B, int *C, int k, int p)
{
    int *dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1);
 
    operate<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C, k, p);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void operate(int *A, int* B, int *C, int k, int p)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x;

    for(int i = tid_x; i < N; i+= gridDim.x * blockDim.x)
        C[i] = operation(A[i], B[i], k, p);
}

In [ ]:
#@title Compile and run
filepath = "zadatak6.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "188 99" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 7

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji računa sledeći izraz: A + B, gde su A i B kvadratne matrice. U novodobijenoj matrici sve vrednosti koje su veće od nekog k zameniti brojem p. Brojeve k i p unosi korisnik. Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja programa u zavisnosti od broja blokova i broja niti u okviru jednog bloka.

%%writefile zadatak7.cu

#include <cuda.h>
#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32

__host__ void operate_on_GPU(int *A, int* B, int* C, int k, int p);
__global__ void operate(int *A, int* B, int *C, int k, int p);
__host__ bool check_result(int *A, int *B, int *C, int k, int p);
__device__ __host__ int operation(int a, int b, int k, int p) { return a + b > k ? p : a + b; }

int main(int argc, char** argv)
{
    int* A, *B, *C, k, p;
    A = (int*) malloc(sizeof(int) * N * N);
    B = (int*) malloc(sizeof(int) * N * N);
    C = (int*) malloc(sizeof(int) * N * N);
  
    if(argc == 3) {
        k = atoi(argv[1]);
        p = atoi(argv[2]);
    }
    else {
        k = 17090;
        p = 2904;
    }
 
    for(int i = 0; i < N; i++) {
      for(int j = 0; j < N; j++) {
        A[i * N + j] = i * N + j;
        B[i * N + j] = j * N + i;
      }
    }
 
    operate_on_GPU(A, B, C, k, p);

    for(int i = 0; i < N; i++) {
        printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", C[i * N + j]);
      printf("|\n");
    }
 
    if(check_result(A, B, C, k, p))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *C, int k, int p)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (C[i * N + j] == operation(A[i * N + j], B[i * N + j], k, p));
    return rez;
}

__host__ void operate_on_GPU(int *A, int *B, int *C, int k, int p)
{
    int* dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.y + 1);
 
    operate<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C, k, p);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void operate(int *A, int* B, int *C, int k, int p)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x,
        tid_y = blockIdx.y * blockDim.y + threadIdx.y;

    for(int i = tid_x; i < N; i+= gridDim.x * blockDim.x)
      for(int j = tid_y; j < N; j += gridDim.y * blockDim.y)
        C[i * N + j] = operation(A[i * N + j], B[i * N + j], k, p);
}

In [ ]:
#@title Compile and run
filepath = "zadatak7.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "17190 2904" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 8

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji računa sledeći izraz: A - B, gde su A i B kvadratne matrice. U novodobijenoj matrici sve vrednosti koje su manje od nekog k zameniti brojem p. Brojeve k i p unosi korisnik. Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja programa u zavisnosti od broja blokova i broja niti u okviru jednog bloka.

%%writefile zadatak8.cu

#include <cuda.h>
#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32

__host__ void operate_on_GPU(int *A, int* B, int* C, int k, int p);
__global__ void operate(int *A, int* B, int *C, int k, int p);
__host__ bool check_result(int *A, int *B, int *C, int k, int p);
__device__ __host__ int operation(int a, int b, int k, int p) { return a - b < k ? p : a - b; }

int main(int argc, char** argv)
{
    int* A, *B, *C, k, p;
    A = (int*) malloc(sizeof(int) * N * N);
    B = (int*) malloc(sizeof(int) * N * N);
    C = (int*) malloc(sizeof(int) * N * N);
  
    if(argc == 3) {
        k = atoi(argv[1]);
        p = atoi(argv[2]);
    }
    else {
        k = 17090;
        p = 2904;
    }
 
    for(int i = 0; i < N; i++) {
      for(int j = 0; j < N; j++) {
        A[i * N + j] = i * N + j;
        B[i * N + j] = j * N + i;
      }
    }
 
    operate_on_GPU(A, B, C, k, p);

    for(int i = 0; i < N; i++) {
        printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", C[i * N + j]);
      printf("|\n");
    }
 
    if(check_result(A, B, C, k, p))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *C, int k, int p)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (C[i * N + j] == operation(A[i * N + j], B[i * N + j], k, p));
    return rez;
}

__host__ void operate_on_GPU(int *A, int *B, int *C, int k, int p)
{
    int* dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.y + 1);
 
    operate<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C, k, p);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void operate(int *A, int* B, int *C, int k, int p)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x,
        tid_y = blockIdx.y * blockDim.y + threadIdx.y;

    for(int i = tid_x; i < N; i+= gridDim.x * blockDim.x)
      for(int j = tid_y; j < N; j += gridDim.y * blockDim.y)
        C[i * N + j] = operation(A[i * N + j], B[i * N + j], k, p);
}

In [ ]:
#@title Compile and run
filepath = "zadatak8.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 9

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji računa sledeći izraz: A*x, gde je A kvadratna matrica, a x skalar. Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja programa u zavisnosti od broja blokova i broja niti u okviru jednog bloka.

%%writefile zadatak9.cu

#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32

__const__ int x = 5;

__host__ void operate_on_GPU(int *A, int *C);
__global__ void operate(int *A, int * C);
__host__ bool check_result(int *A, int *C);
__device__ __host__ int operation(int a) { return a * x; }

int main(int argc, char** argv)
{
    int* A, *C;
    A = (int*) malloc(sizeof(int) * N * N);
    C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++)
      for(int j = 0; j < N; j++)
        A[i * N + j] = j * N + i;
 
    operate_on_GPU(A, C);

    for(int i = 0; i < N; i++) {
        printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", C[i * N + j]);
      printf("|\n");
    }
 
    if(check_result(A, C))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(C);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *C)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (C[i * N + j] == operation(A[i * N + j]));
    return rez;
}

__host__ void operate_on_GPU(int *A, int *C)
{
    int* dev_A, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.y + 1);
 
    operate<<<gridSize, blockSize>>>(dev_A, dev_C);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_C);
}

__global__ void operate(int *A, int* C)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x,
        tid_y = blockIdx.y * blockDim.y + threadIdx.y;
        
    for(int i = tid_x; i < N; i+= gridDim.x * blockDim.x)
      for(int j = tid_y; j < N; j += gridDim.y * blockDim.y)
        C[i * N + j] = operation(A[i * N + j]);
}

In [ ]:
#@title Compile and run
filepath = "zadatak9.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 10

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji broji broj ponavljanja elementa k u kvadratnoj matrici A. Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja programa u zavisnosti od broja blokova i broja niti u okviru jednog bloka.

%%writefile zadatak10.cu

#define N 512
#define BLOCK_SIZE 32

__const__ int k = 5;


__global__ void operate_on_GPU(int *A, int *counter)
{
    int tid_x = blockIdx.x * blockDim.x + threadId.x,
        tid_y = blockIdx.y * blockDim.y + threadId.y;
 
    __shared__ int local_counter[BLOCK_SIZE * BLOCK_SIZE];
 
    local_counter[tid_x * BLOCK_SIZE + tid_x] = tid_x < N && tid_y < N ? A[tid_x * N + tid_y] == k ? 1 : 0 : 0;

    __syncthreads();
 
    for(int i = )

    if(threadId.x == 0 && threadId.y == 0)
    counter[blockIdx.x] = local_counter[0];
}

int main(int argc, char** argv)
{
    return 0;
}

In [ ]:
#@title Compile and run
filepath = "zadatak10.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 11

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji za data dva vektora A i B kreira treći, vektor C tako da je c[i] = max(A[i], B[i]). Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja sekvencijalnog i paralelnog programa.

%%writefile zadatak11.cu

#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32

__host__ void operate_on_GPU(int *a, int *b, int *c);
__global__ void operate(int *a, int *b, int *c);
__host__ bool check_result(int *a, int *b, int *gpu);
__device__ __host__ int operation(int a, int b) { return max(a, b); }

int main(int argc, char** argv)
{
    int *a, *b, *c;
    a = (int*) malloc(sizeof(int) * N);
    b = (int*) malloc(sizeof(int) * N);
    c = (int*) malloc(sizeof(int) * N);
 
    for(int i = 0; i < N; i++)
      a[i] = N - (b[i] = i);
 
    operate_on_GPU(a, b, c);

    printf("c = |\t");
    for(int i = 0; i < N; i++)
      printf("%d\t", c[i]);
    printf("|\n");
 
    if(check_result(a, b, c))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(c);
    free(b);
    free(a);

    return 0;
}

__host__ bool check_result(int *a, int *b, int *gpu)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
        rez &= (gpu[i] == operation(a[i], b[i]));
    return rez;
}

__host__ void operate_on_GPU(int *a, int *b, int *c)
{
    int *dev_a, *dev_b, *dev_c;
 
    cudaMalloc((void**) &dev_a, sizeof(int) * N);
    cudaMalloc((void**) &dev_b, sizeof(int) * N);
    cudaMalloc((void**) &dev_c, sizeof(int) * N);
 
    cudaMemcpy(dev_a, a, sizeof(int) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, sizeof(int) * N, cudaMemcpyHostToDevice);
 
    operate<<<N / BLOCK_SIZE + 1, BLOCK_SIZE>>>(dev_a, dev_b, dev_c);
 
    cudaMemcpy(c, dev_c, sizeof(int) * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);
}

__global__ void operate(int *a, int *b, int *c)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x;

    for(int i = tid_x; i < N; i += gridDim.x * blockDim.x)
      c[i] = operation(a[i], b[i]);
}

In [ ]:
#@title Compile and run
filepath = "zadatak11.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 12

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji za data dva vektora A i B kreira treći, vektor C tako da je c[i] = min(A[i], B[i]). Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja sekvencijalnog i paralelnog programa.

%%writefile zadatak12.cu

#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32

__host__ void operate_on_GPU(int *a, int *b, int *c);
__global__ void operate(int *a, int *b, int *c);
__host__ bool check_result(int *a, int *b, int *gpu);
__device__ __host__ int operation(int a, int b) { return min(a, b); }

int main(int argc, char** argv)
{
    int *a, *b, *c;
    a = (int*) malloc(sizeof(int) * N);
    b = (int*) malloc(sizeof(int) * N);
    c = (int*) malloc(sizeof(int) * N);
 
    for(int i = 0; i < N; i++)
      a[i] = N - (b[i] = i);
 
    operate_on_GPU(a, b, c);

    printf("c = |\t");
    for(int i = 0; i < N; i++)
      printf("%d\t", c[i]);
    printf("|\n");
 
    if(check_result(a, b, c))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(c);
    free(b);
    free(a);

    return 0;
}

__host__ bool check_result(int *a, int *b, int *gpu)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
        rez &= (gpu[i] == operation(a[i], b[i]));
    return rez;
}

__host__ void operate_on_GPU(int *a, int *b, int *c)
{
    int *dev_a, *dev_b, *dev_c;
 
    cudaMalloc((void**) &dev_a, sizeof(int) * N);
    cudaMalloc((void**) &dev_b, sizeof(int) * N);
    cudaMalloc((void**) &dev_c, sizeof(int) * N);
 
    cudaMemcpy(dev_a, a, sizeof(int) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, sizeof(int) * N, cudaMemcpyHostToDevice);
 
    operate<<<N / BLOCK_SIZE + 1, BLOCK_SIZE>>>(dev_a, dev_b, dev_c);
 
    cudaMemcpy(c, dev_c, sizeof(int) * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);
}

__global__ void operate(int *a, int *b, int *c)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x;

    for(int i = tid_x; i < N; i += gridDim.x * blockDim.x)
      c[i] = operation(a[i], b[i]);
}

In [ ]:
#@title Compile and run
filepath = "zadatak12.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 13

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji za date dve kvadratne matrice A i B kreira treću, matricu C tako da je c[i][j] = max(A[i][j], B[i][j]). Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja sekvencijalnog i paralelnog programa.

%%writefile zadatak13.cu

#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32

__host__ void operate_on_GPU(int *A, int *B, int *C);
__global__ void operate(int *A, int *B, int *C);
__host__ bool check_result(int *A, int *B, int *GPU);
__device__ __host__ int operation(int a, int b) { return max(a, b); }

int main(int argc, char** argv)
{
    int *A, *B, *C;
    A = (int*) malloc(sizeof(int) * N * N);
    B = (int*) malloc(sizeof(int) * N * N);
    C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++)
      for(int j = 0; j < N; j++)
        A[i * N + j] = (N * N) - (B[i * N + j] = i * N + j);
 
    operate_on_GPU(A, B, C);

    for(int i = 0; i < N; i++) {
      printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", C[i * N + j]);
      printf("|\n");
    }
 
    if(check_result(A, B, C))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *GPU)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (GPU[i * N + j] == operation(A[i * N + j], B[i * N + j]));
    return rez;
}

__host__ void operate_on_GPU(int *A, int *B, int *C)
{
    int *dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / BLOCK_SIZE + 1, N / BLOCK_SIZE + 1);
 
    operate<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void operate(int *A, int *B, int *C)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x,
        tid_y = blockIdx.y * blockDim.y + threadIdx.y;

    for(int i = tid_x; i < N; i += gridDim.x * blockDim.x)
      for(int j = tid_y; j < N; j += gridDim.y * blockDim.y)
        C[i * N + j] = operation(A[i * N + j], B[i * N + j]);
}

In [ ]:
#@title Compile and run
filepath = "zadatak13.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 14

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji za date dve kvadratne matrice A i B kreira treću, matricu C tako da je c[i][j] = min(A[i][j], B[i][j]). Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja sekvencijalnog i paralelnog programa.

%%writefile zadatak14.cu

#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32

__host__ void operate_on_GPU(int *A, int *B, int *C);
__global__ void operate(int *A, int *B, int *C);
__host__ bool check_result(int *A, int *B, int *GPU);
__device__ __host__ int operation(int a, int b) { return min(a, b); }

int main(int argc, char** argv)
{
    int *A, *B, *C;
    A = (int*) malloc(sizeof(int) * N * N);
    B = (int*) malloc(sizeof(int) * N * N);
    C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++)
      for(int j = 0; j < N; j++)
        A[i * N + j] = (N * N) - (B[i * N + j] = i * N + j);
 
    operate_on_GPU(A, B, C);

    for(int i = 0; i < N; i++) {
      printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", C[i * N + j]);
      printf("|\n");
    }
 
    if(check_result(A, B, C))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *GPU)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (GPU[i * N + j] == operation(A[i * N + j], B[i * N + j]));
    return rez;
}

__host__ void operate_on_GPU(int *A, int *B, int *C)
{
    int *dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / BLOCK_SIZE + 1, N / BLOCK_SIZE + 1);
 
    operate<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void operate(int *A, int *B, int *C)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x,
        tid_y = blockIdx.y * blockDim.y + threadIdx.y;

    for(int i = tid_x; i < N; i += gridDim.x * blockDim.x)
      for(int j = tid_y; j < N; j += gridDim.y * blockDim.y)
        C[i * N + j] = operation(A[i * N + j], B[i * N + j]);
}

In [ ]:
#@title Compile and run
filepath = "zadatak14.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 15

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji računa sledeći izraz: A - B, gde su A i B vektori. Sve negativne vrednosti zameniti nulom. Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja programa u zavisnosti od broja blokova i broja niti u okviru jednog bloka.

%%writefile zadatak15.cu

#include <stdio.h>
#define N 5120
#define BLOCK_SIZE 32

__host__ void operate_on_GPU(int *A, int *B, int *C);
__global__ void operate(int *A, int *B, int *C);
__host__ bool check_result(int *A, int *B, int *GPU_C);
__device__ __host__ int operation(int a, int b) { return max(a - b, 0); }

int main(int argc, char** argv)
{
    int *A, *B, *GPU_C;
    A = (int*) malloc(sizeof(int) * N);
    B = (int*) malloc(sizeof(int) * N);
    GPU_C = (int*) malloc(sizeof(int) * N);
 
    for(int i = 0; i < N; i++) {
        A[i] = i * i;
        B[i] = N - (i + 1);
    }
 
    operate_on_GPU(A, B, GPU_C);
 
    printf("c = |\t");
    for(int i = 0; i < N; i++)
      printf("%d\t", GPU_C[i]);
    printf("|\n");
 
    if(check_result(A, B, GPU_C))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat je netacan!\n");
 
    free(GPU_C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *GPU_C)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
        rez &= (GPU_C[i] == operation(A[i], B[i]));
    return rez;
}

__host__ void operate_on_GPU(int *A, int *B, int *C)
{
    int *dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1);
 
    operate<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void operate(int *A, int* B, int *C)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x;

    for(int i = tid_x; i < N; i+= gridDim.x * blockDim.x)
        C[i] = operation(A[i], B[i]);
}

In [ ]:
#@title Compile and run
filepath = "zadatak15.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "" #@param [] { allow-input: true }

!./$compiled_filepath $argv

# Zadatak 16

In [ ]:
#@title Code
#@markdown Napisati CUDA program koji računa sledeći izraz: A - B, gde su A i B kvadratne matrice. Sve negativne vrednosti zameniti nulom. Napisati kod za testiranje validnosti rezultata, upoređivanjem sa vrednostima dobijenim izvršavanjem sekvencijalnog koda koji izračunava isti izraz. Pripremiti se za diskusiju ponašanja programa u zavisnosti od broja blokova i broja niti u okviru jednog bloka.
%%writefile zadatak16.cu

#include <cuda.h>
#include <stdio.h>
#define N 512
#define BLOCK_SIZE 32

__host__ void operate_on_GPU(int *A, int* B, int* C);
__global__ void operate(int *A, int* B, int *C);
__host__ bool check_result(int *A, int *B, int *C);
__device__ __host__ int operation(int a, int b) { return max(a - b, 0); }

int main(int argc, char** argv)
{
    int* A, *B, *C;
    A = (int*) malloc(sizeof(int) * N * N);
    B = (int*) malloc(sizeof(int) * N * N);
    C = (int*) malloc(sizeof(int) * N * N);
 
    for(int i = 0; i < N; i++) {
      for(int j = 0; j < N; j++) {
        A[i * N + j] = i * N + j;
        B[i * N + j] = j * N + i;
      }
    }
 
    operate_on_GPU(A, B, C);

    for(int i = 0; i < N; i++) {
        printf("|\t");
      for(int j = 0; j < N; j++)
        printf("%d\t", C[i * N + j]);
      printf("|\n");
    }
 
    if(check_result(A, B, C))
      printf("Rezultat je tacan!\n");
    else
      printf("Rezultat nije tacan!\n");
 
    free(C);
    free(B);
    free(A);

    return 0;
}

__host__ bool check_result(int *A, int *B, int *C)
{
    bool rez = true;
    for(int i = 0; i < N && rez; i++)
      for(int j = 0; j < N && rez; j++)
        rez &= (C[i * N + j] == operation(A[i * N + j], B[i * N + j]));
    return rez;
}

__host__ void operate_on_GPU(int *A, int *B, int *C)
{
    int* dev_A, *dev_B, *dev_C;
 
    cudaMalloc((void**) &dev_A, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_B, sizeof(int) * N * N);
    cudaMalloc((void**) &dev_C, sizeof(int) * N * N);
 
    cudaMemcpy(dev_A, A, sizeof(int) * N * N, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, sizeof(int) * N * N, cudaMemcpyHostToDevice);
 
    dim3 blockSize(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridSize(N / blockSize.x + 1, N / blockSize.y + 1);
 
    operate<<<gridSize, blockSize>>>(dev_A, dev_B, dev_C);
 
    cudaMemcpy(C, dev_C, sizeof(int) * N * N, cudaMemcpyDeviceToHost);

    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
}

__global__ void operate(int *A, int* B, int *C)
{
    int tid_x = blockIdx.x * blockDim.x + threadIdx.x,
        tid_y = blockIdx.y * blockDim.y + threadIdx.y;

    for(int i = tid_x; i < N; i+= gridDim.x * blockDim.x)
      for(int j = tid_y; j < N; j += gridDim.y * blockDim.y)
        C[i * N + j] = operation(A[i * N + j], B[i * N + j]);
}

In [ ]:
#@title Compile and run
filepath = "zadatak16.cu"  #@param { type: "string" }
compiled_filepath = "ime_kompajlirane_datoteke"  #@param { type: "string" }

!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 $filepath -o $compiled_filepath 

argv = "" #@param [] { allow-input: true }

!./$compiled_filepath $argv